# **저시력자를 위한 원화 화폐 분류**
---
- 본 과제는 UltraLytics YOLO v5 모델 사용을 권장합니다.
    - 본 파일의 목차는 UltraLytics YOLO v5에 맞게 작성되어 있습니다.
    - 다른 모델을 찾아서 사용하셔도 좋습니다.
    - 산출물이 잘 나오면 됩니다 : )
---

## 0.미션
---
- **과제 수행 목표**
    - 본 과제는 Object Detection 문제입니다.
    - Object Detection 문제로 접근하기 위해 **데이터셋 전처리**를 하셔야 합니다.
    - 데이터셋 : money_dataset.zip
        1. 데이터셋은 압축 파일로 제공됩니다.
        2. 압축 파일 안에는 화폐마다 폴더가 개별적으로 존재합니다.
        3. 폴더 안에는 화폐 이미지와 화폐 정보가 담긴 json 파일이 있습니다.
    - 여러분이 직접 촬영한 화폐 사진들을 탐지 과정에서 이용 해보세요.
    - 이미지에 화폐 하나만 나오게 촬영하는 것은 지양해주세요.
    - 다양한 방법으로 화폐를 촬영하고 결과를 확인해보세요.
        - ex 1) 화폐의 모든 종류를 한 이미지에 나오게 촬영
        - ex 2) 여러 화폐를 겹치게 하여 촬영
---
- **Key Point**
    1. 모델에 맞는 폴더 구조 확인
    2. 이미지 축소 비율에 맞춰 좌표값 변경
        - 좌표를 이미지 리사이즈한 비율로 변경
    3. 모델에 맞는 정보 추출/형식 변경
        - json 파일에서 정보 추출 및 모델 형식에 맞게 변경
    4. 화폐당 하나의 클래스로 변경
        - 총 8개 클래스
    5. 모델 선택 필요
---

## 1.환경설정

### (1) 구글 드라이브 연동, 데이터 다운로드
---
- 아래의 코드 셀을 반드시 실행시켜야 합니다.
---

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install gdown

### (2) 데이터셋 불러오기
---
- **세부요구사항**
    - 데이터셋 파일의 압축을 해제하세요.
---
- 예제 코드에서는 zipfile 모듈을 이용하였습니다.
    - [zipfile document](https://docs.python.org/3/library/zipfile.html#zipfile-objects)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [3]:
import zipfile, gdown,os
url ="https://drive.google.com/file/d/1k1tXDK35s6BsMTPGWSl5GVGNoPfC898X/view?usp=drive_link"
file_name = "money_dataset.zip"
output = "/content/drive/MyDrive/" + file_name # 변경 가능
if not os.path.exists(output):
    gdown.download(url=url, output=output, quiet=False, fuzzy=True)

In [4]:
# 데이터셋 압축 파일 경로 : 유저별로 상이할 수 있음
money_data = zipfile.ZipFile(output)

In [5]:
# 데이터셋 압축 해제
money_data.extractall('/content/Dataset/')

## 2.데이터 전처리

### (1) 폴더 구조 생성 및 파일 이동
---
- **세부요구사항**
    -  모델에서 요구하는 폴더 구조를 만들어야 합니다.
        - Hint : Image와 Label을 구분하는 폴더를 만들어 주세요
---
- 예제 코드에서는 glob, shutil 모듈을 이용하였습니다.
    - [glob document](https://docs.python.org/3/library/glob.html) | [shutil document](https://docs.python.org/3/library/shutil.html)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [6]:
# 1.폴더 구조 만들기
!mkdir /content/Dataset/images;
!mkdir /content/Dataset/images/train; mkdir /content/Dataset/images/val

!mkdir /content/Dataset/labels;
!mkdir /content/Dataset/labels/train; mkdir /content/Dataset/labels/val

In [7]:
import glob, shutil

In [8]:
# 2. Dataset metadata 입력
won_list = ['10', '50', '100', '500', '1000', '5000', '10000', '50000']
data_path = '/content/Dataset/'

---
- 데이터를 Training set | Validation set으로 분할하세요.
    - 예시 : Training과 Validation은 8:2로 분리
- Hint : 이미지 데이터는 /images에, JSON 데이터는 /labels에 넣어주세요
    - 예시 : /dataset/images/train, /dataset/labels/train
    - 예제 코드에서는 glob, shutil 모듈을 이용하였습니다.
    - [glob document](https://docs.python.org/3/library/glob.html) | [shutil document](https://docs.python.org/3/library/shutil.html)

    ※ 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
    
---

In [ ]:
########################
# 이 셀부터 코드 작성하세요
########################
# 3. 데이터를 Training set | Validation set으로 분할하세요.


In [9]:
import os
from sklearn.model_selection import train_test_split

# image_file과 json_file명 list에 저장
image_files = [[]]*len(won_list)

images = [[]]*len(won_list)
for x in range(len(won_list)):
  image_files[x] = glob.glob(data_path+won_list[x]+"/*.jpg")


In [10]:
# 이미지와 json 확장자를 임시로 제거 ()
for x in range(len(won_list)):
  images[x] = [name.replace(".jpg","") for name in image_files[x]]

In [11]:
# 경로 설정
data_path = '/content/Dataset/'
train_dir = '/content/Dataset/images/train'
test_dir = '/content/Dataset/images/val'

In [12]:
from sklearn.model_selection import train_test_split
train_names = [[]]*len(images)
test_names = [[]]*len(images)
for x in range(len(images)):
  train_names[x], test_names[x] = train_test_split(images[x], test_size=0.2, random_state=42)

In [13]:
import os
def batch_move_files(file_list, source_path, destination_path):
    for file in file_list:
        image = file.split('/')[-1] + '.jpg'
        txt = file.split('/')[-1] + '.json'
        shutil.copy(os.path.join(source_path, image), destination_path)
        shutil.copy(os.path.join(source_path, txt), destination_path)
    return

In [14]:
for x in range(len(train_names)):
  batch_move_files(train_names[x], data_path+won_list[x],train_dir)
for x in range(len(test_names)):
  batch_move_files(test_names[x], data_path+won_list[x],test_dir)

In [15]:
json_train_dir = glob.glob('/content/Dataset/images/train/*.json')
json_test_dir = glob.glob('/content/Dataset/images/val/*.json')

In [198]:
shutil.rmtree('/content/Dataset/labels/train/')
shutil.rmtree('/content/Dataset/labels/val/')

In [220]:
#shutil.rmtree('/content/yolov5/runs/val')

In [16]:
# 1.폴더 구조 만들기
!mkdir /content/Dataset/images;
!mkdir /content/Dataset/images/train; mkdir /content/Dataset/images/val

!mkdir /content/Dataset/labels;
!mkdir /content/Dataset/labels/train; mkdir /content/Dataset/labels/val

mkdir: cannot create directory ‘/content/Dataset/images’: File exists
mkdir: cannot create directory ‘/content/Dataset/images/train’: File exists
mkdir: cannot create directory ‘/content/Dataset/images/val’: File exists
mkdir: cannot create directory ‘/content/Dataset/labels’: File exists
mkdir: cannot create directory ‘/content/Dataset/labels/train’: File exists
mkdir: cannot create directory ‘/content/Dataset/labels/val’: File exists


In [17]:
target_train_dir = '/content/Dataset/labels/train'
target_test_dir = '/content/Dataset/labels/val'
for x in range(len(json_train_dir)):
  shutil.move(json_train_dir[x], target_train_dir)
for x in range(len(json_test_dir)):
  shutil.move(json_test_dir[x], target_test_dir)

### (2) json에서 정보 추출
---
- **세부요구사항**
    - json 파일에서 필요한 정보를 추출하세요:
        - 위치 정보 : x1, x2, y1, y2
        - 박스 정보 : shape_type
        - 클래스 정보 : labels
    - 화폐당 하나의 클래스로 변경하세요.
        - json 파일에는 화폐 클래스가 앞뒷면으로 구분되어 있습니다.
        - 화폐의 앞뒷면 구분을 없애주세요.
            - 예시 : 'ten_front', 'ten_back' -> 'ten'
    - 화폐의 위치 정보를 YOLO 모델 형식에 맞게 변경 해주세요.
        - 사용되는 이미지는 원본에서 1/5로 축소되어 있습니다.
        - json 파일의 정보는 원본 기준 데이터이므로 위치 정보 추출을 할 때 x값과 y값을 1/5로 줄여주세요.
    - 이렇게 변경된 정보를 YOLO label 형식에 맞게 txt파일로 저장 해 주세요.
        - Hint : YOLO Labeling Format [label, x-center, y-center, width-norm, height-norm]
---

In [18]:
import os, json

In [19]:
json_path = '/content/Dataset/labels/'
temp_list = ['train', 'val']

In [20]:
########################
# 이 셀부터 코드 작성하세요
# Json 파일에서 필요한 정보만 골라 txt로 바꾸는 작업임을 기억하세요!
########################

with open('/content/Dataset/labels/val/10000_B_DESK_0_105.json') as file:
  datas = json.load(file)

  json_t = datas['shapes'][0]['points']
  print(json_t)

[[68.69872439999999, 535.583088], [1046.0451672, 996.1148316]]


In [22]:

input_dir = "/content/Dataset/labels/val"
output_dir = "/content/Dataset/labels/val"
label_mapping = {
    #10
    'Ten_back': '0',
    'Ten_front' : '0',
    #50
    'Fifty_back': '1',
    'Fifty_front' : '1',
    #100
    'Hundred_back': '2',
    'Hundred_front' : '2',
    #500
    'Five_Hundred_back': '3',
    'Five_Hundred_front' : '3',
    #1000
    'Thousand_back': '4',
    'Thousand_front' : '4',
    #5000
    'Five_Thousand_back': '5',
    'Five_Thousand_front' : '5',
    #10000
    "Ten_Thousand_back": "6",
    'Ten_Thousand_front' : '6',
    #50000
    "Fifty_Thousand_back": "7",
    'Fifty_Thousand_front' : '7',
}

In [23]:
for filename in os.listdir(input_dir):
    if filename.endswith(".json"):
        # JSON 파일 읽기
        json_file_path = os.path.join(input_dir, filename)
        with open(json_file_path, "r") as json_file:
            data = json.load(json_file)
            image_width = data["imageWidth"] / 5
            image_height = data["imageHeight"] / 5
            shapes = data["shapes"]

            # 결과를 저장할 리스트 초기화
            results = []

            for shape in shapes:
                # 위치 정보 추출 및 1/5 크기로 조정
                points = shape["points"]
                x1, y1 = points[0]
                x2, y2 = points[1]
                x1 /= 5
                y1 /= 5
                x2 /= 5
                y2 /= 5

                # 박스 정보 추출
                shape_type = shape["shape_type"]

                # 클래스 정보 추출 및 매핑
                original_label = shape["label"]
                mapped_label = label_mapping.get(original_label, original_label)

                # YOLO 형식에 맞게 변환
                x_center = (x1 + x2) / 2 / image_width
                y_center = (y1 + y2) / 2 / image_height
                width_norm = (x2 - x1) / image_width
                height_norm = (y2 - y1) / image_height

                # YOLO 형식으로 변환된 정보를 리스트에 추가
                results.append([mapped_label, x_center, y_center, width_norm, height_norm])

        # 결과를 TXT 파일로 저장
        output_txt_file_path = os.path.join(output_dir, filename.replace(".json", ".txt"))
        with open(output_txt_file_path, "w") as output_txt_file:
            for result in results:
                line = " ".join(str(x) for x in result)
                output_txt_file.write(line + "\n")

### (3) 데이터셋 정보가 담긴 파일 생성
---
- **세부요구사항**
    - 파일 안에 있어야 할 정보는 아래와 같습니다.
        - 학습할 클래스 이름 정보
        - 학습할 클래스 수 정보
        - Training, Validation 데이터셋 위치 정보
---
- 가장 대중적으로 이용하는 라이브러리는 yaml 입니다.
    - [yaml document](https://pyyaml.org/wiki/PyYAMLDocumentation)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [24]:
import yaml

In [25]:
won_dict = {0:'10', 1:'50', 2:'100', 3:'500', 4:'1000', 5:'5000', 6:'10000', 7:'50000'}
t_dic = ['10', '50','100', '500','1000','5000', '10000','50000']

In [25]:
########################
# 이 셀부터 코드 작성하세요
########################

with open('/content/Dataset/data.yaml', 'r') as f :
  yaml_data = yaml.load(f, Loader=yaml.FullLoader)
  display(yaml_data)


FileNotFoundError: ignored

In [26]:
num_classes = len(won_dict)
training_dataset_path = "/content/Dataset/images/train"
validation_dataset_path = "/content/Dataset/images/val"

yaml_data = {
    "names": won_dict,
    "nc": num_classes,
    "train": training_dataset_path,
    "val": validation_dataset_path
}

# YAML 파일로 데이터 저장
output_yaml_file_path = "/content/Dataset/data.yaml"

with open(output_yaml_file_path, "w") as output_yaml_file:
    yaml.dump(yaml_data, output_yaml_file)

In [27]:
import os

def count_empty_txt_files(directory):
    empty_txt_count = 0  # 빈 txt 파일 개수를 저장할 변수 초기화

    # 주어진 디렉토리 내의 모든 파일 및 하위 디렉토리 검색
    for root, dirs, files in os.walk(directory):
        for filename in files:
            # 파일 확장자가 ".txt"인지 확인
            if filename.endswith(".txt"):
                file_path = os.path.join(root, filename)
                # 파일의 크기가 0바이트인 경우 (빈 파일)
                if os.path.getsize(file_path) == 0:
                    empty_txt_count += 1

    return empty_txt_count

# 디렉토리 경로 설정 (원하는 디렉토리 경로로 변경)
directory_path = "/content/Dataset/labels/val"

# 빈 txt 파일 개수 계산
count = count_empty_txt_files(directory_path)

# 결과 출력
print(f"빈 txt 파일 개수: {count}")

빈 txt 파일 개수: 0


## 3.모델링

### (1) 모델 라이브러리 설치
---

In [28]:
!git clone https://github.com/ultralytics/yolov5  # clone

!pip install -r yolov5/requirements.txt  # install

Cloning into 'yolov5'...
remote: Enumerating objects: 15994, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 15994 (delta 18), reused 18 (delta 12), pack-reused 15967
Receiving objects: 100% (15994/15994), 14.58 MiB | 23.55 MiB/s, done.
Resolving deltas: 100% (10984/10984), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.5/189.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.1/618.1 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00


### (2) 가중치 파일 다운로드
---
- **세부요구사항**
    - 모델 개발자가 제공하는 사전 학습 가중치 파일을 다운로드 하세요.
        - 해당 과정이 불필요하다면 넘어가셔도 됩니다!
---

In [29]:
!chmod +w /content/Dataset/labels/
!chmod +w /content/Dataset/

### (3) 학습 : train.py
---
- **세부요구사항**
    - UltraLytics YOLO v5에는 아래의 데이터가 필요합니다.
        - 데이터셋 정보가 담긴 yaml 파일
        - 사용하려는 모델 구조에 대한 yaml 파일
        - 사용하려는 모델의 가중치 파일
---

In [34]:
########################
# 이 셀부터 코드 작성하세요
########################
%cd /content/yolov5/

!python train.py --epochs 7 --data /content/Dataset/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name yolov5s_results

/content/yolov5
train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=/content/Dataset/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=7, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-224-g6262c7f Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0,

## 4.탐지 : detect.py
---
- **세부요구사항**
    - 학습 과정에서 생성된 가중치 파일을 이용하세요.
    - IoU threshold를 0.25 이하로 설정하세요.
    - confidence threshold를 0.75 이상으로 설정하세요.
---
- 여러분이 **직접 촬영한 화폐 사진과 동영상**을 탐지 과정에 이용하여 결과를 확인하세요.
    - 조건
        1. 화폐의 수를 늘려가며 촬영 해보세요.
            - ex) 50원 하나, 50원 둘, 50원 셋, ...
        2. 화폐의 종류를 늘려가며 촬영 해보세요.
            - ex) 50원 하나와 100원 하나, 50원 하나와 100원 하나와 1000원 하나, ...
        3. 사진은 최소 30장 이상, 동영상은 최소 하나 이상 촬영하여 사용 해보세요.
---

In [42]:
########################
# 이 셀부터 코드 작성하세요
########################
!python detect.py --source /content/test/c1.jpg --weights /content/yolov5/runs/train/yolov5s_results5/weights/best.pt --iou-thres 0.25 --conf-thres 0.75

detect: weights=['/content/yolov5/runs/train/yolov5s_results5/weights/best.pt'], source=/content/test/c1.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.75, iou_thres=0.25, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-224-g6262c7f Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/test/c1.jpg: 480x640 (no detections), 44.5ms
Speed: 0.5ms pre-process, 44.5ms inference, 0.4ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp7
